### In this notebook, we present how to query the ES index for paper classification.

We assume that the ES is working and responding at the http://localhost:{`PORT`},
and an index with name defined in `IDX_NAME` is already build.

In [1]:
from config import IDX_NAME, PORT
from source.env_setup.setup import connect_elasticsearch

Check if the `IDX_NAME` is defined and print its structure.

In [2]:
with connect_elasticsearch() as es:
    mappings = es.indices.get_mapping(index=IDX_NAME)

Yay Connected


In [3]:
import json
print(json.dumps(mappings, sort_keys=True, indent=4))

{
    "ontology_index": {
        "mappings": {
            "properties": {
                "broader": {
                    "type": "text"
                },
                "closeMatch": {
                    "type": "text"
                },
                "name": {
                    "type": "text"
                },
                "prefLabel": {
                    "type": "text"
                },
                "related": {
                    "type": "text"
                },
                "type": {
                    "type": "text"
                }
            }
        }
    }
}


Preparing data - sample articles:

In [4]:
from source.data.sample_dataset import get_data


articles = get_data()['data']

In [5]:
print(f"Number of articles: {len(articles)}")

Number of articles: 4


Sample article:

In [6]:
a0 = articles[0]
a0

{'article_title': 'Brain Tumor Detection from MRI using Adaptive Thresholding and Histogram based Techniques',
 'article_abstract': 'This paper depicts a computerized framework that can distinguish brain tumor and investigate the diverse highlights of the tumor. Brain tumor segmentation means to isolated the unique tumor tissues, for example, active cells, edema and necrotic center from ordinary mind tissues of WM, GM, and CSF. However, manual segmentation in magnetic resonance data is a timeconsuming task. We present a method of automatic tumor segmentation in magnetic resonance images which consists of several steps. The recommended framework is helped by image processing based technique that gives improved precision rate of the cerebrum tumor location along with the computation of tumor measure. In this paper, the location of brain tumor from MRI is recognized utilizing adaptive thresholding with a level set and a morphological procedure with histogram. Automatic brain tumor stage i

Let's see how querying the index works! 

A comprehensive guide for the ES queries can be found at: https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-multi-match-query.html

In [7]:
from source.es_index.query_index import find_n_best
from source.result_saving.result_vocabulary import save_result_vocabulary

First, we will feed only the tile.

In [10]:
query = {'query': {
    "match": {"prefLabel" : a0['article_title']}
    }
}

with connect_elasticsearch() as es:
    res = find_n_best(es, IDX_NAME, query, n=10, label_colname='prefLabel')

print(res)

Yay Connected
[{'prefLabel': ['Balanced histogram thresholding'], 'score': 14.568794}, {'prefLabel': ['Tumor detection'], 'score': 14.40885}, {'prefLabel': ['Brain tumor segmentation'], 'score': 14.073767}, {'prefLabel': ['Adaptive histogram equalization'], 'score': 12.219425}, {'prefLabel': ['Thresholding'], 'score': 11.079433}, {'prefLabel': ['Mri segmentation'], 'score': 10.983144}, {'prefLabel': ['Histogram'], 'score': 10.140241}, {'prefLabel': ['Blocking techniques'], 'score': 9.020447}, {'prefLabel': ['Recovery techniques'], 'score': 9.020447}, {'prefLabel': ['Visualisation techniques'], 'score': 9.020447}]


We can try `multi_match`

In [8]:
query = {
  "query": {
    "multi_match" : {
      "query":    a0['article_title'], 
      "fields" : ["prefLabel^3", "related", "broader"]
    }
  }
}

with connect_elasticsearch() as es:
    res = find_n_best(es, IDX_NAME, query, n=10, label_colname='prefLabel')

print(res)

Yay Connected
[{'prefLabel': ['Balanced histogram thresholding'], 'score': 43.706383}, {'prefLabel': ['Tumor detection'], 'score': 43.226555}, {'prefLabel': ['Brain tumor segmentation'], 'score': 42.221302}, {'prefLabel': ['Adaptive histogram equalization'], 'score': 36.658276}, {'prefLabel': ['Thresholding'], 'score': 33.2383}, {'prefLabel': ['Mri segmentation'], 'score': 32.949432}, {'prefLabel': ['Histogram'], 'score': 30.420723}, {'prefLabel': ['Blocking techniques'], 'score': 27.061342}, {'prefLabel': ['Recovery techniques'], 'score': 27.061342}, {'prefLabel': ['Visualisation techniques'], 'score': 27.061342}]


In [9]:
print(save_result_vocabulary(query, res))

@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/result/90a9daef-69c6-413a-ab2f-0762a89b4f5f> a schema:SearchAction ;
    schema:endTime "2023-01-11"^^xsd:date ;
    schema:query "{'query': {'multi_match': {'query': 'Brain Tumor Detection from MRI using Adaptive Thresholding and Histogram based Techniques', 'fields': ['prefLabel^3', 'related', 'broader']}}}" ;
    schema:result "[{'prefLabel': ['Balanced histogram thresholding'], 'score': 43.706383}, {'prefLabel': ['Tumor detection'], 'score': 43.226555}, {'prefLabel': ['Brain tumor segmentation'], 'score': 42.221302}, {'prefLabel': ['Adaptive histogram equalization'], 'score': 36.658276}, {'prefLabel': ['Thresholding'], 'score': 33.2383}, {'prefLabel': ['Mri segmentation'], 'score': 32.949432}, {'prefLabel': ['Histogram'], 'score': 30.420723}, {'prefLabel': ['Blocking techniques'], 'score': 27.061342}, {'prefLabel': ['Recovery techniques'], 'score': 27.061342}, {'prefLabel'

In [12]:
query = {
  "query": {
    "multi_match" : {
      "query":      a0['article_title'],
      "type":       "bool_prefix",
      "fields":     ["prefLabel^4", "related", "broader^2"]
    }
  }
}

with connect_elasticsearch() as es:
    res = find_n_best(es, IDX_NAME, query, n=10, label_colname='prefLabel')

print(res)

Yay Connected
[{'prefLabel': ['Balanced histogram thresholding'], 'score': 99.951416}, {'prefLabel': ['Adaptive histogram equalization'], 'score': 87.61258}, {'prefLabel': ['Histogram matching'], 'score': 66.12162}, {'prefLabel': ['Iterative thresholding'], 'score': 63.40815}, {'prefLabel': ['Thresholding algorithm'], 'score': 63.40815}, {'prefLabel': ['Recurrent Gastrointestinal Stromal Tumor'], 'score': 58.001457}, {'prefLabel': ['Tumor detection'], 'score': 57.6354}, {'prefLabel': ['China brain'], 'score': 57.433777}, {'prefLabel': ['Brain tumor segmentation'], 'score': 56.295067}, {'prefLabel': ['Thresholding'], 'score': 54.596306}]


Now, let's add the abstract as well.

In [9]:
query = {
  "query": {
    "dis_max": {
      "queries": [
        {
          "multi_match" : {
          "query":    a0['article_title'], 
          "fields" : ["prefLabel^3", "related", "broader"]
    }
        },
        {
          "multi_match" : {
          "query":    a0['article_abstract'], 
          "fields" : ["prefLabel^2", "related", "broader^2"]
          }
        }
      ]
    }
  }
}

with connect_elasticsearch() as es:
    res = find_n_best(es, IDX_NAME, query, n=10, label_colname='prefLabel')

print(res)

Yay Connected
[{'prefLabel': ['Brain tumor segmentation'], 'score': 244.49348}, {'prefLabel': ['Liver tumor segmentation'], 'score': 234.81906}, {'prefLabel': ['Tumor segmentation'], 'score': 207.3719}, {'prefLabel': ['Stromal Neoplasm'], 'score': 195.96886}, {'prefLabel': ['Small Intestinal Gastrointestinal Stromal Tumor'], 'score': 195.96886}, {'prefLabel': ['Malignant Gastrointestinal Stromal Tumor'], 'score': 195.96886}, {'prefLabel': ['Gastric Gastrointestinal Stromal Tumor'], 'score': 195.96886}, {'prefLabel': ['Esophageal GIST'], 'score': 195.96886}, {'prefLabel': ['Recurrent Gastrointestinal Stromal Tumor'], 'score': 195.96886}, {'prefLabel': ['Duodenal submucosal tumor'], 'score': 195.96886}]


We can try different parameters in the Elastic Search, e.g., `best_fields` query type.

In [33]:
query = {
  "query": {
    "dis_max": {
      "queries": [
        {
          "multi_match" : {
          "query":      a0['article_title'],
          "type":       "best_fields",
          "fields":     ["prefLabel^3", "related", "broader"],
          "tie_breaker": 0.3
         }
        },
        {
          "multi_match" : {
          "query":      a0['article_abstract'],
          "type":       "best_fields",
          "fields":     ["prefLabel^3", "related", "broader"],
          "tie_breaker": 0.3
         }
        }
      ]
    }
  }
}

with connect_elasticsearch() as es:
    query_res = find_n_best(es, IDX_NAME, query, n=10, label_colname='prefLabel')

print(query_res)

Yay Connected
[{'prefLabel': ['Brain tumor segmentation'], 'score': 374.2478}, {'prefLabel': ['Tumor segmentation'], 'score': 318.56543}, {'prefLabel': ['Liver tumor segmentation'], 'score': 297.5333}, {'prefLabel': ['Recurrent Gastrointestinal Stromal Tumor'], 'score': 255.27765}, {'prefLabel': ['Tumor detection'], 'score': 254.29216}, {'prefLabel': ['Tumor region'], 'score': 254.29216}, {'prefLabel': ['Nerve tumor'], 'score': 253.00272}, {'prefLabel': ['Stromal tumor'], 'score': 253.00272}, {'prefLabel': ['Is-a'], 'score': 246.47375}, {'prefLabel': ['Duodenal submucosal tumor'], 'score': 242.74872}]


In [13]:
query = {
  "query": {
    "dis_max": {
      "queries": [
        {
          "multi_match" : {
          "query":      a0['article_title'],
          "type":       "most_fields",
          "fields":     ["prefLabel^3", "related", "broader"],
          "tie_breaker": 0.5
         }
        },
        {
          "multi_match" : {
          "query":      a0['article_abstract'],
          "type":       "most_fields",
          "fields":     ["prefLabel^3", "related", "broader"],
          "tie_breaker": 0.5
         }
        }
      ]
    }
  }
}

with connect_elasticsearch() as es:
    res = find_n_best(es, IDX_NAME, query, n=10, label_colname='prefLabel')

print(res)

Yay Connected
[{'prefLabel': ['Brain tumor segmentation'], 'score': 379.25287}, {'prefLabel': ['Tumor segmentation'], 'score': 323.5705}, {'prefLabel': ['Liver tumor segmentation'], 'score': 321.0152}, {'prefLabel': ['Recurrent Gastrointestinal Stromal Tumor'], 'score': 302.49762}, {'prefLabel': ['Duodenal submucosal tumor'], 'score': 262.3456}, {'prefLabel': ['Gastrointestinal stroma tumor'], 'score': 262.3456}, {'prefLabel': ['Tumor detection'], 'score': 255.1518}, {'prefLabel': ['Tumor region'], 'score': 255.1518}, {'prefLabel': ['Is-a'], 'score': 254.2988}, {'prefLabel': ['Nerve tumor'], 'score': 253.00272}]
